In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import diagnostic
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.model_selection import cross_validate

In [ ]:
pd.options.display.max_columns = None

### Loading and wrangling data

In [ ]:
PATH = './data_tesco/'

GROCERY_WARD = 'year_osward_grocery.csv'
GROCERY_BOROUGH = 'year_borough_grocery.csv'

DIABETES_WARD = 'diabetes_estimates_osward_2016.csv'
OBESITY_BOROUGH = 'london_obesity_borough_2012.csv'
CHILD_OBESITY_WARD = 'child_obesity_london_ward_2013-2014.csv'

MAPPING_BOROUGH_WARD = 'Mapping-template-london-ward-map-2018.xls'
MAPPING_CODE_BOROUGH = 'Mapping-template-for-London-boroughs.xls'

BOUNDARIES_BOROUGH = 'statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'

CONSUMER_EXPENDITURE_BOROUGH = 'detailed-borough-base.xls'
CHILDREN_POVERTY_BOROUGH = 'children-in-poverty.xls'
EARNINGS_BOROUGH = 'earnings-residence-borough.xlsx'

#### Grocery data

In [ ]:
groceries_borough = pd.read_csv(PATH + GROCERY_BOROUGH, sep = ',', header = 0)
groceries_ward = pd.read_csv(PATH + GROCERY_WARD, sep = ',', header = 0)
# Consider representativeness?
groceries_borough = groceries_borough[groceries_borough['representativeness_norm'] >= 0.1]
groceries_ward = groceries_ward[groceries_ward['representativeness_norm'] >= 0.1]

#### Ward to borough mapping

In [ ]:
mapping_ward_to_borough = pd.read_excel(MAPPING_BOROUGH_WARD, sheet_name = 'Ward Thematic Map',usecols = [0, 1, 2],
                                        names = ['Ward Code', 'Ward name', 'Borough name'])
mapping_code_to_borough = pd.read_excel(MAPPING_CODE_BOROUGH, sheet_name = 'Borough Thematic Map',usecols = [1, 2],
                                       names = ['Borough Code', 'Borough name'])
mapping_ward_to_borough = mapping_ward_to_borough.merge(mapping_code_to_borough, on = 'Borough name')
mapping_ward_to_borough = mapping_ward_to_borough.drop(columns = ['Ward name', 'Borough name'])

#### Health data

Some of the data (diabetes) are only available for borough, thus we average all wards part of a same borough and map the data to the corresponding borough.

In [ ]:
# Diabetes
diabetes_ward = pd.read_csv(PATH + DIABETES_WARD, sep = ',', header = 0)
diabetes_ward = diabetes_ward.merge(mapping_ward_to_borough, left_on = 'area_id', right_on = 'Ward Code')
diabetes_ward = diabetes_ward.drop(columns = ['Ward Code'])
diabetes_borough = diabetes_ward.groupby(['Borough Code'], as_index = False).mean()
len(diabetes_borough)

In [ ]:
# Obesity
obesity_borough = pd.read_csv(PATH + OBESITY_BOROUGH, sep = ',', header = 0)
len(obesity_borough)

In [ ]:
# Child obesity ward
child_obesity_ward = pd.read_csv(PATH + CHILD_OBESITY_WARD, sep = ',', header = 0)
to_drop = child_obesity_ward[ (child_obesity_ward['prevalence_overweight_reception'] == 'na')
                             | (child_obesity_ward['prevalence_overweight_y6'] == 'na')
                            | (child_obesity_ward['prevalence_obese_reception'] == 'na')
                            | (child_obesity_ward['prevalence_obese_y6'] == 'na')].index
child_obesity_ward.drop(to_drop , inplace=True)
child_obesity_ward = child_obesity_ward.merge(mapping_ward_to_borough, left_on = 'area_id', right_on = 'Ward Code')
child_obesity_ward = child_obesity_ward.drop(columns = ['Ward Code'])
child_obesity_ward = child_obesity_ward.astype({'number_reception_measured': 'float64',
          'number_y6_measured': 'float64',
          'prevalence_overweight_reception': 'float64',
          'prevalence_overweight_y6': 'float64',
          'prevalence_obese_reception': 'float64',
          'prevalence_obese_y6': 'float64'})
child_obesity_borough = child_obesity_ward.groupby(['Borough Code'], as_index = False).mean()
len(child_obesity_borough)

Merge data for wards

In [ ]:
merged_ward = pd.merge(diabetes_ward, groceries_ward, how = 'inner', left_index = False, on='area_id')
merged_ward.head()

Merge data for boroughs

In [ ]:
merged = pd.merge(groceries_borough, obesity_borough, how = 'inner', left_index = False, left_on='area_id',
                         right_on = 'oslaua')
merged = pd.merge(merged, child_obesity_borough, how = 'inner', left_index = False, left_on='area_id',
                         right_on = 'Borough Code')
merged_borough = pd.merge(merged, diabetes_borough, how = 'inner', left_index = False, left_on='area_id',
                         right_on = 'Borough Code')
print(len(merged_borough))
merged_borough.head()

Load and merge boundaries

In [ ]:
map_borough = gpd.read_file(BOUNDARIES_BOROUGH)
merged = map_borough.merge(merged_borough, left_on = 'GSS_CODE', right_on = 'area_id', how = 'inner')

#### Economic data

In [ ]:
children_poverty = pd.read_excel(PATH+CHILDREN_POVERTY_BOROUGH, sheet_name = '2015',usecols = [0, 5],
                                        names = ['area_id', 'child_poverty'])
children_poverty = children_poverty.loc[21:53]
children_poverty = children_poverty.astype({'child_poverty': 'float64'})

In [ ]:
earnings = pd.read_excel(PATH+EARNINGS_BOROUGH, sheet_name = 'Full-time, Weekly',usecols = [1, 28],
                                       names = ['Borough name', 'earnings'], na_values=['#'])
earnings = earnings.loc[2:34]
earnings = earnings.astype({'earnings': 'float64'})

In [ ]:
consumer_expenditure = pd.read_excel(PATH+CONSUMER_EXPENDITURE_BOROUGH, sheet_name = 'Greater London',usecols = [0, 1, 17],
                                        names = ['Borough name', 'type', 'expenditure'], skiprows=[0, 1, 2])
consumer_expenditure.dropna(inplace=True)
consumer_expenditure.sort_values(by=['Borough name'], inplace=True)
consumer_expenditure = consumer_expenditure.pivot_table(values='expenditure', index='Borough name', columns='type')
consumer_expenditure.head()

In [ ]:
consumer_expenditure = consumer_expenditure.transform(lambda x: (x / x.sum()), axis=1)

### Visualize prevalence of diabetes and obesity/overweight

Check if there's a correlation between the indicators

In [ ]:
indicators = ['estimated_diabetes_prevalence', 'prevalence_obese_reception', 'prevalence_obese_y6','f_obese', 'prevalence_overweight_reception', 'prevalence_overweight_y6', 'f_overweight']
df = merged_borough[indicators]

mask = np.triu(np.ones_like(df.corr(), dtype=np.bool))
heatmap = sns.heatmap(df.corr(method='spearman'), vmin=-1, vmax=1, mask = mask, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12});

Check if those indicators are correlated with any food habits

In [ ]:
# We partition 'types' of food habits
energy_nutrients = ['energy_tot', 'energy_fat', 'energy_saturate', 'energy_sugar', 'energy_protein', 'energy_carb', 
                    'energy_fibre', 'energy_alcohol'] #'h_nutrients_calories'
f_food_categories = ['f_beer', 'f_dairy', 'f_eggs', 'f_fats_oils', 'f_fish', 'f_fruit_veg', 'f_grains',
                   'f_meat_red', 'f_poultry', 'f_readymade', 'f_sauces', 'f_soft_drinks', 'f_spirits',
                   'f_sweets', 'f_tea_coffee', 'f_water', 'f_wine']
features = energy_nutrients + f_food_categories
indicators = ['estimated_diabetes_prevalence', 'f_overweight']

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(8, 8))

for i, ind in enumerate(indicators):
    labels = features.copy()
    labels.append(ind)
    df = merged_borough[labels]
    heatmap = sns.heatmap(df.corr(method='spearman')[[ind]].sort_values(by = ind, ascending=False), vmin=-1, vmax=1, annot=True, ax=axs[i])
    heatmap.set_title(ind, fontdict={'fontsize':12})
    
plt.subplots_adjust(wspace=1)

#### Fancy plot sur le notebook de Mama

### Visualize average diet

Plot nutrient and category repartition in average diet

In [ ]:
average_nutrients = pd.DataFrame(groceries_ward.mean(axis = 0)[energy_nutrients], columns = ['nutrient'])
average_categories = pd.DataFrame(groceries_ward.mean(axis = 0)[f_food_categories], columns = ['category'])

In [ ]:
nutrients_labels = ['Energy tot', 'Energy fat', 'Energy saturate', 'Energy sugar', 'Energy protein', 'Energy carb', 
                    'Energy fibre', 'Energy alcohol']
food_labels = ['Beer', 'Dairy', 'Eggs', 'Fats & oils', 'Fish', 'Fruit & veg', 'Grains',
                   'Red meat', 'Poultry', 'Readymade', 'Sauces', 'Soft drinks', 'Spirits',
                   'Sweets', 'Tea & coffee', 'Water', 'Wine']

In [ ]:
plot = go.Figure(data=[go.Pie( 
    name='Nutrients', 
    values=average_nutrients['nutrient'], 
    labels=nutrients_labels,
    marker_colors=px.colors.sequential.Viridis,
    textposition='inside',
    hoverinfo='percent+label',
    insidetextorientation='auto',
    hole=0.5
), 
    go.Pie( 
    name='Food', 
    values=average_categories['category'], 
    labels=food_labels,
    marker_colors=px.colors.sequential.Viridis,
    textposition='inside',
    hoverinfo='percent+label',
    insidetextorientation='auto',
    hole=0.5
) 
]) 
  
plot.update_layout( 
    updatemenus=[ 
        dict( 
            active=0, 
            buttons=list([  
                dict(label="Nutrient categories", 
                     method="update", 
                     args=[{"visible": [True, False]}, 
                           {"title": "Londoners Average Diet", 
                            }]), 
                dict(label="Food categories", 
                     method="update", 
                     args=[{"visible": [False, True]}, 
                           {"title": "Londoners Average Diet", 
                            }]), 
            ]), 
        ) 
    ]) 
  
plot.write_html("figures/piecharts.html")    
plot.show() 

### Compare the average diet to the WHO recommendations

We compute 5 criterias that make a diet healthy and that are either 0 or 1 depending on if they are fulfilled or not: fat criteria (< 30% of total energy intake), saturated fat criteria (< 10% of total energy intake), sugars criteria (< 10% of total energy intake), etc... We obtain a final score between 0 and 5, out of 5.

In [ ]:
# Total fat should not exceed 30% of total energy intake
groceries_borough['WHO_totalfat'] = 0
groceries_borough.loc[(groceries_borough['energy_fat'] <= 0.3*groceries_borough['energy_tot']), 'WHO_totalfat'] = 1
groceries_borough['WHO_totalfat'].sum() / groceries_borough.shape[0]

In [ ]:
# Intake of saturated fats should be less than 10% of total energy intake
groceries_borough['WHO_saturatedfat'] = 0
groceries_borough.loc[(groceries_borough['energy_saturate'] <= 0.1*groceries_borough['energy_tot']), 'WHO_saturatedfat'] = 1
groceries_borough['WHO_saturatedfat'].sum() / groceries_borough.shape[0]

In [ ]:
# Intake of free sugars should be less than 10% of total energy intake
groceries_borough['WHO_freesugars'] = 0
groceries_borough.loc[(groceries_borough['energy_fat'] <= 0.1*groceries_borough['energy_sugar']), 'WHO_freesugars'] = 1
groceries_borough['WHO_freesugars'].sum() / groceries_borough.shape[0]

In [ ]:
# c
groceries_borough['WHO_salt'] = 0
groceries_borough.loc[(groceries_borough['salt'] <= 5), 'WHO_salt'] = 1
groceries_borough['WHO_salt'].sum() / groceries_borough.shape[0]

The London boroughs only fulfill the salt criterion from the WHO recommendations.

#### Fancy visualization à base de flipping cards?

### Compute a diet score

#### Method 1 : fit a linear regression on the overweight and obesity data with the highest correlated energy_nutrients features

In [ ]:
indicators = ['prevalence_obese_reception', 'prevalence_obese_y6', 'f_obese', 'prevalence_overweight_reception', 'prevalence_overweight_y6', 'f_overweight']

In [ ]:
for i in indicators:
    correlation = pd.DataFrame(columns = ['feature', 'R', 'p_value'])
    for f in energy_nutrients:
        corr = stats.spearmanr(merged_borough[i], merged_borough[f])
        corr_data = pd.DataFrame([[f, corr[0], corr[1]]], columns = ['feature', 'R', 'p_value'])
        correlation = correlation.append(corr_data, ignore_index = True)
    sig_features = correlation[correlation['p_value'] < 0.05]
    print(i)
    print(sig_features)

We only select the features with a statistically significant Spearman rank correlation (p < 0.05) with almost two indicators.

In [ ]:
features = ['energy_carb', 'energy_fibre', 'h_nutrients_calories']

In [ ]:
def find_weights(merged, label):
    X = merged[features]
    y = merged[label]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    hyperparams = {'reg':[0.1, 0.01, 0.001, 0.0001], 'learning_rate':[0.1, 0.05, 0.01]}
    scenarios = []
    for lr in hyperparams['learning_rate']:
        for r in hyperparams['reg']:
            scores = cross_validate(SGDRegressor(alpha = r, eta0 = lr),X, y = y, cv = 3,
                             scoring=('explained_variance', 'neg_mean_squared_error', 'r2'))
            scenario = {'learning_rate':lr, 'regularizer':r,
                        'MSE': scores['test_neg_mean_squared_error'],
                        'Explained variance': scores['test_explained_variance'],
                       'r2' : scores['test_r2']}
            #print(scores)
            scenarios.append(scenario)

    best = np.argmax([np.mean(s['r2']) for s in scenarios])

    reg_opt = scenarios[best]['regularizer']
    lr_opt = scenarios[best]['learning_rate']
    print(reg_opt, lr_opt)
    model_opt = SGDRegressor(eta0 = lr_opt, alpha = reg_opt)
    model_opt.fit(X,y)
    r2 = model_opt.score(X, y)
    train_error = np.mean((model_opt.predict(X)-y)**2)
    print(label)
    print(("Training error : {}, R2 : {}").
          format(train_error, r2))
    return model_opt.coef_

In [ ]:
weights = pd.DataFrame(index = features, columns = indicators)

for label in indicators:
    weights[label] = find_weights(merged_borough, label)

In [ ]:
weights = weights.mean(axis = 1)

In [ ]:
def score(area):
    df = area[features].values
    score = (df*weights).sum()
    return score

In [ ]:
groceries_borough['score1'] = groceries_borough.apply(score, axis=1)
scaler = MinMaxScaler()
groceries_borough['score1'] = 1 - scaler.fit_transform(groceries_borough[['score1']].values.reshape(-1,1))

Check consistency : plot repartition of 25% lowest and 25% highest scoring areas

In [ ]:
lowest = pd.DataFrame(groceries_borough[groceries_borough['score1'] <= groceries_borough['score1'].quantile(0.25)][features]).T
highest = pd.DataFrame(groceries_borough[groceries_borough['score1'] >= groceries_borough['score1'].quantile(0.75)][features]).T
highest

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False, figsize=(14, 4))

for i, f in enumerate(features):
    axs[i].bar([0, 1], [lowest.loc[f].mean(), highest.loc[f].mean()], color=['tab:orange', 'tab:green'])
    axs[i].set_xticks([0, 1])
    axs[i].set_xticklabels(['unhealthiest', 'healthiest'])
    axs[i].set_title(f)

In [ ]:
titles = ["Energy carb", "Energy fibre", "Entropy nutrients"]
fig = make_subplots(rows=1, cols=3, horizontal_spacing=0.05, vertical_spacing=0.1,
                   subplot_titles=(titles[0], titles[1], titles[2]))

def add_bar_subplot(f, col):
    fig.add_trace(go.Bar(
        x=[0, 1], 
        y=[lowest.loc[f].mean(), highest.loc[f].mean()], 
        marker_color=['#ff7f0e', '#2ca02c'],
        hovertemplate = titles[col] + ": %{y:.3f}<br>" + "<extra></extra>"),
        row=1, col=col+1)
    fig['layout']['xaxis'+str(col+1)].update(tickvals=[0,1], ticktext=['unhealthy', 'healthy'])

                  
for i, f in enumerate(features):
    add_bar_subplot(f, i)

fig.update_layout(height=400, width=800, showlegend=False)
fig.write_html("figures/barplots_score1.html")
fig.show()

#### Method 2 : fit a linear regression on the obesity datasets with the most consumed f_food_categories features

In [ ]:
features = ['f_fruit_veg', 'f_sweets', 'f_grains', 'f_dairy']

In [ ]:
weights = pd.DataFrame(index = features, columns = indicators)

for label in indicators:
    weights[label] = find_weights(merged_borough, label)

In [ ]:
weights = weights.mean(axis = 1)

In [ ]:
groceries_borough['score2'] = groceries_borough.apply(score, axis=1)
scaler = MinMaxScaler()
groceries_borough['score2'] = 1 - scaler.fit_transform(groceries_borough[['score2']].values.reshape(-1,1))

Problem : Optimal hyperparameters keep changing through runs. Maybe due to small data?
Also, for now we select best hyper-param according to r2 score. should maybe change?

In [ ]:
lowest = pd.DataFrame(groceries_borough[groceries_borough['score2'] <= groceries_borough['score2'].quantile(0.25)][features]).T
highest = pd.DataFrame(groceries_borough[groceries_borough['score2'] >= groceries_borough['score2'].quantile(0.75)][features]).T
highest

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, sharex=False, sharey=False, figsize=(14, 4))

for i, f in enumerate(features):
    axs[i].bar([0, 1], [lowest.loc[f].mean(), highest.loc[f].mean()], color=['tab:orange', 'tab:green'])
    axs[i].set_xticks([0, 1])
    axs[i].set_xticklabels(['unhealthiest', 'healthiest'])
    axs[i].set_title(f)

In [ ]:
titles = ['Fruit & veg', 'Sweets', 'Grains', 'Dairy']
fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.05, vertical_spacing=0.1,
                   subplot_titles=(titles[0], titles[1], titles[2], titles[3]))

def add_bar_subplot(f, col):
    fig.add_trace(go.Bar(
        x=[0, 1], 
        y=[lowest.loc[f].mean(), highest.loc[f].mean()], 
        marker_color=['#ff7f0e', '#2ca02c'],
        hovertemplate = titles[col] + ": %{y:.3f}<br>" + "<extra></extra>"),
        row=1, col=col+1)
    fig['layout']['xaxis'+str(col+1)].update(tickvals=[0,1], ticktext=['unhealthy', 'healthy'])

                  
for i, f in enumerate(features):
    add_bar_subplot(f, i)

fig.update_layout(height=400, width=800, showlegend=False)
fig.write_html("figures/barplots_score2.html")
fig.show()

### Map visualization of the two scores

In [ ]:
def plot_london(variable, merged_borough):
    MAPBOX_ACCESSTOKEN = 'pk.eyJ1Ijoic29zbzk0IiwiYSI6ImNraTdwem80MDFsNXEyc3FzeGMxOHpoZGkifQ.coIFQU-pN6pZJi0GuXnLVw'
    merged = map_borough.merge(merged_borough, left_on = 'GSS_CODE', right_on = 'area_id', how = 'inner')
    merged = merged.to_crs(epsg=4326) # convert the coordinate reference system to lat/long
    lga_json = merged.__geo_interface__ #covert to geoJSON
    zmin = merged[variable].min()
    zmax = merged[variable].max()
    name = merged['NAME']

    # Set the data for the map
    data = go.Choroplethmapbox(
            geojson = lga_json,             #this is your GeoJSON
            locations = merged.index,    #the index of this dataframe should align with the 'id' element in your geojson
            z = merged[variable], #sets the color value
            text = merged.NAME,    #sets text for each shape
            colorbar=dict(thickness=20, ticklen=3, tickformat='',outlinewidth=0), #adjusts the format of the colorbar
            marker_line_width=1, marker_opacity=0.7, colorscale="Viridis", #adjust format of the plot
            zmin=zmin, zmax=zmax,           #sets min and max of the colorbar
            hovertemplate = "<b>%{text}</b><br>" +
                             "%{z:.0%}<br>" +
                            "<extra></extra>")  # sets the format of the text shown when you hover over each shape

    # Set the layout for the map
    layout = go.Layout(     #format the plot title
        mapbox1 = dict(
            domain = {'x': [0, 1],'y': [0, 1]}, 
            center = dict(lat=51.509865 , lon=-0.118092),
            accesstoken = MAPBOX_ACCESSTOKEN, 
            zoom = 8.8),                      
        autosize=True,
        height=650,
        margin=dict(l=0, r=0, t=40, b=0))

    # Generate the map
    data, layout
    fig=go.Figure(data=data, layout=layout)
    fig.write_image("figures/" + variable + "_map.jpeg")
    fig.show()

In [ ]:
plot_london('score1', groceries_borough)

In [ ]:
plot_london('score2', groceries_borough)

### Validation of the two score and selection of the best one

In [ ]:
merged_borough = pd.merge(merged_borough, groceries_borough[['area_id', 'score1', 'score2']], on='area_id')

In [ ]:
corr = merged_borough[['estimated_diabetes_prevalence', 'score1', 'score2']].corr(method='spearman')
corr

In [ ]:
features = ['score1', 'score2', 'estimated_diabetes_prevalence']
features_name = ['Score 1', 'Score 2', 'Diabetes prevalence']
colors = ['#3e4989', '#1f9e89', '#b5de2b']
titles = [f'Spearman rank correlation = {corr.iloc[1,2]:.3f}',
         f'Spearman rank correlation = {corr.iloc[0,1]:.3f}',
         f'Spearman rank correlation = {corr.iloc[0,2]:.3f}']

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.1, vertical_spacing=0.1,
                specs=[[{"colspan": 2}, None], [{}, {}]],
                   subplot_titles=(titles[0], titles[1], titles[2]))

def add_scatter_subplot(f1, f2, row, col):
    fig.add_trace(go.Scatter(
        x=merged_borough[features[f1]], 
        y=merged_borough[features[f2]],
        marker_color = colors[row+col-2],
        mode='markers',
        hovertemplate = features_name[f1] + ": %{x:.3f}<br>" + features_name[f2] + ": %{y:.3f}<br>" + "<extra></extra>"),
        row=row, col=col)
    fig['layout']['xaxis'+str(row+col-1)].update(title_text=features_name[f1], title_standoff=3)
    fig['layout']['yaxis'+str(row+col-1)].update(title_text=features_name[f2], title_standoff=3)
    
add_scatter_subplot(0, 1, 1, 1)
add_scatter_subplot(0, 2, 2, 1)
add_scatter_subplot(1, 2, 2, 2)

fig.update_layout(height=800, width=800, showlegend=False)
fig.write_html("figures/scatter2D.html")
fig.show()

As score1 has a biggest Spearman rank correlation coefficient than score2, we will only use score1 in our future analyses.

### Relate the diet score to economic factors

Merge the economic data

In [ ]:
groceries_poverty = pd.merge(groceries_borough, children_poverty, how = 'inner', on='area_id')

boroughs = mapping_code_to_borough.copy()
boroughs = pd.merge(boroughs, earnings, how='inner', on='Borough name')
groceries_earnings = pd.merge(groceries_borough, boroughs, how = 'inner', left_on='area_id', right_on='Borough Code')

boroughs = mapping_code_to_borough.copy()
boroughs = pd.merge(boroughs, consumer_expenditure['Food'], how='inner', on='Borough name')
groceries_expenditure = pd.merge(groceries_borough, boroughs, how = 'inner', left_on='area_id', right_on='Borough Code')
groceries_expenditure.rename(columns={'Food':'food_expenditure'}, inplace=True)

In [ ]:
merged_borough = pd.merge(merged_borough, groceries_poverty[['area_id', 'child_poverty']], on='area_id')
merged_borough = pd.merge(merged_borough, groceries_earnings[['area_id', 'earnings']], on='area_id')
merged_borough = pd.merge(merged_borough, groceries_expenditure[['area_id', 'food_expenditure']], on='area_id')
merged_borough.head()

London Consumer Expenditure Estimates - Detailed Borough Base: consumer expenditure data to 2036 broken down by London borough. We will transform the data concerning food expenditure in percentage of the total expenditure over the year 2015.

In [ ]:
def plot_london_data(variable, merged_borough, hovertemplate):
    merged = map_borough.merge(merged_borough, left_on = 'GSS_CODE', right_on = 'area_id', how = 'inner')
    merged = merged.to_crs(epsg=4326) # convert the coordinate reference system to lat/long
    lga_json = merged.__geo_interface__ #covert to geoJSON
    zmin = merged[variable].min()
    zmax = merged[variable].max()
    name = merged['NAME']

    # Set the data for the map
    data = go.Choroplethmapbox(
            geojson = lga_json,             #this is your GeoJSON
            locations = merged.index,    #the index of this dataframe should align with the 'id' element in your geojson
            z = merged[variable], #sets the color value
            text = merged.NAME,    #sets text for each shape
            colorbar=dict(thickness=20, ticklen=3, tickformat='',outlinewidth=0), #adjusts the format of the colorbar
            marker_line_width=1, marker_opacity=0.7, colorscale="Viridis", #adjust format of the plot
            zmin=zmin, zmax=zmax,           #sets min and max of the colorbar
            hovertemplate=hovertemplate)  # sets the format of the text shown when you hover over each shape
    return data

In [ ]:
def plot_london_layout():
    MAPBOX_ACCESSTOKEN = 'pk.eyJ1Ijoic29zbzk0IiwiYSI6ImNraTdwem80MDFsNXEyc3FzeGMxOHpoZGkifQ.coIFQU-pN6pZJi0GuXnLVw'
    
    # Set the layout for the map
    layout = go.Layout(
        title = {'font': {'size':24}},  
        mapbox1 = dict(
            domain = {'x': [0, 1],'y': [0, 1]}, 
            center = dict(lat=51.509865 , lon=-0.118092),
            accesstoken = MAPBOX_ACCESSTOKEN, 
            zoom = 8.8),                      
        autosize=True,
        height=650,
        margin=dict(l=0, r=0, t=40, b=0))

    return layout

In [ ]:
plot = go.Figure(data=[plot_london_data('earnings', groceries_earnings, "<b>%{text}</b><br>"+"%{z:.0f}£<br>"+"<extra></extra>"), 
        plot_london_data('child_poverty', groceries_poverty, "<b>%{text}</b><br>"+"%{z:.0f}£<br>"+"<extra></extra>"),
          plot_london_data('food_expenditure', groceries_expenditure, "<b>%{text}</b><br>"+"%{z:.0%}<br>"+"<extra></extra>")],
       layout = plot_london_layout())

plot.update_layout( 
    updatemenus=[ 
        dict( 
            active=0, 
            buttons=list([  
                dict(label="Earnings", 
                     method="update", 
                     args=[{"visible": [True, False, False]}, 
                           {"title": "London economic indicators", 
                            }]), 
                dict(label="Child poverty", 
                     method="update", 
                     args=[{"visible": [False, True, False]}, 
                           {"title": "London economic indicators", 
                            }]),
                dict(label="Food expenditure", 
                     method="update", 
                     args=[{"visible": [False, False, True]}, 
                           {"title": "London economic indicators", 
                            }])
            ]), 
        ) 
    ]) 
  
plot.write_html("figures/economicindicators_map.html")    
plot.show() 

In [ ]:
features = merged_borough[['score1', 'child_poverty', 'earnings', 'food_expenditure']]
features.corr(method='spearman')

In [ ]:
features = ['child_poverty', 'earnings', 'food_expenditure','avg_age']
weights = find_weights(merged_borough, 'score1')

In [ ]:
fig = px.scatter_3d(merged_borough, x='food_expenditure', y='child_poverty', z='earnings',
              color='score1', color_continuous_scale=px.colors.sequential.Viridis, width=600, height=500,
            labels={"food_expenditure": "Food expenditure",
             "child_poverty": "Child poverty (£)",
             "earnings": "Earnings (£)",
             "score1": "Score 1"})
plot.write_html("figures/scatter3d.html")
fig.show()

### Embedding as features for score3?

In [ ]:
f_food_categories = ['energy_tot', 'energy_fat', 'energy_saturate', 'energy_sugar', 'energy_protein', 'energy_carb', 
                    'energy_fibre', 'h_nutrients_calories']

data = np.array([merged_borough['f_beer']]).T
for f in f_food_categories:
    data = np.concatenate((data, np.array([merged_borough[f]]).T), axis=1)

In [ ]:
from sklearn.manifold import TSNE
#from sklearn.decomposition import PCA

data_embedded = TSNE(n_components=3).fit_transform(data)
#pca = PCA(n_components=3)
#data_embedded = pca.fit_transform(data)

data_embedded = pd.DataFrame(data_embedded)
data_embedded.columns = ['x', 'y', 'z']
data_embedded

In [ ]:
fig = px.scatter_3d(data_embedded, x='x', y='y', z='z')
fig.show()